<a href="https://colab.research.google.com/github/bejon23/Apr19_Saimon_Heart_Disease_Pred/blob/main/Bejon_gene_expression_pca_kmeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = ':https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1868%2F3249%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240925%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240925T232154Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8caa7d04febf7940dbdb1fb1bac8b9fabf6a2a9380f94622bbb77581451faf88ba92b9e6c79683db3594c75c425fdb2b90b1366b9d2b2ecaa65a79fc5893d250f2e68e6e6b5d0e70927be89b970bb31c98e664d3f3f63dfff542948945b665f568edb4f2953f205bed420c86164f7ded5634498bda6c56be615a902a8fcbb3cb31fb2afc5e4cc02a3f371f1642dc4cebb8d962de9f361ea22e7a0b85f361a26fb81df035b8250808da560e6c042d7f51690ad7a2b3bfdbeeaeeaa52bdf5c092952a9b45f43a8068bc5c4f962f22d96a635f8409c31c975ccde462725b650a14929b5d3ebc331153029d8b3526329fc2667edaee03a664de4b675ff416d08debb'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/gene-expression/data_set_ALL_AML_train.csv")
actual = pd.read_csv("/kaggle/input/gene-expression/actual.csv")
independent = pd.read_csv("/kaggle/input/gene-expression/data_set_ALL_AML_independent.csv")
train.head()

In [ ]:
actual.head()

In [ ]:
independent.head()

In [ ]:
train.describe()

In [ ]:
train.columns

In [ ]:
# we don't need call columns so let's remove them

trainreq = [col for col in train.columns if "call" not in col]
train = train[trainreq]
train.head()

In [ ]:
train = train.T
train.head()

In [ ]:
train2 = train.drop(['Gene Description','Gene Accession Number'],axis=0)
train2.index = pd.to_numeric(train2.index)
train2.head()


In [ ]:
train2.sort_index(inplace=True)

In [ ]:
train2.head()

# There are two datasets containing the initial (training, 38 samples) and independent (test, 34 samples) datasets used in the paper. These datasets contain measurements corresponding to ALL and AML samples from Bone Marrow and Peripheral Blood. Intensity values have been re-scaled such that overall intensities for each chip are equivalent.

In [ ]:
train2.shape

In [ ]:
actual['cancer'].value_counts()

In [ ]:
patients = list(actual[:38]['cancer'])
train2['Patient_Category'] = patients
train2.head()

In [ ]:
train2['Patient_Category'] = train2['Patient_Category'].replace({"ALL":0,"AML":1})
train2.head()

In [ ]:
X = train2.drop("Patient_Category",axis=1)
y = train2.Patient_Category

In [ ]:
X.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.impute import SimpleImputer

In [ ]:
sc = StandardScaler()
X_scl = sc.fit_transform(X)

# PCA

In [ ]:
n_components=30
pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X_scl)

In [ ]:
Xpcadf = pd.DataFrame(X_pca)
Xpcadf.columns =['PC'+str(i) for i in range(1,n_components+1)]

Xpcadf['Patient Category'] = patients
print(f"Percent of explained variance with {n_components} Components : ", round(pca.explained_variance_ratio_.sum()*100,2))
Xpcadf.head()

In [ ]:
import plotly.express as px
import seaborn as sns
import plotly.graph_objects as go

In [ ]:
plt.figure(figsize=(16,10))
sns.set_style("darkgrid")
ax = sns.scatterplot(Xpcadf['PC1'],Xpcadf['PC2'],hue=Xpcadf['Patient Category'])

plt.xlabel(f'PC1 ({round(pca.explained_variance_ratio_[0] * 100,1)} %)')
plt.ylabel(f'PC2 ({round(pca.explained_variance_ratio_[1] * 100,1)} %)')
plt.title('overall scatter plot of PC1 and PC2')
plt.grid()
plt.show()

In [ ]:
fig = px.scatter_3d(data_frame=Xpcadf,x='PC1',y='PC2',z='PC3',color='Patient Category',template='plotly_dark')
fig.update_layout(title = "3D Scatter plot for first 3 principal components",title_x=0.5)
fig.show()

In [ ]:
variance = []
comps = []
for comp in range(2,31,1):
    pca = PCA(n_components=comp)
    imppca = pca.fit_transform(X_scl)
    var = round(pca.explained_variance_ratio_.sum()*100,2)
    variance.append(var)
    comps.append(comp)

pcatable = pd.DataFrame({"Number of components":comps,"% of variance explained":variance})
pcatable.style.background_gradient(cmap="Reds")

In [ ]:
plt.figure(figsize=(12,8))
sns.barplot(data=pcatable,x='Number of components',y='% of variance explained',palette='coolwarm')
plt.show()

# Lets find out which all features added the most to our PC1 and PC2

In [ ]:
X.head(2)

In [ ]:
def create_importance_dataframe(pca, original_num_df):
    importance_df  = pd.DataFrame(pca.components_)
    importance_df.columns  = original_num_df.columns
    importance_df =importance_df.apply(np.abs)
    importance_df=importance_df.transpose()
    num_pcs = importance_df.shape[1]
    new_columns = [f'PC{i}' for i in range(1, num_pcs + 1)]
    importance_df.columns  =new_columns
    return importance_df

importance_df  =create_importance_dataframe(pca,X)
display(importance_df.head())

In [ ]:
pc1_top_10_features = importance_df['PC1'].sort_values(ascending = False)[:10]
print(), print(f'PC1 top 10 features are \n')
pd.DataFrame(pc1_top_10_features).reset_index().rename(columns={"index":"Gene","PC1":"Importance value for PC1"})

In [ ]:
pc2_top_10_features = importance_df['PC2'].sort_values(ascending = False)[:10]
print(), print(f'PC2 top 10 features are \n')
pd.DataFrame(pc1_top_10_features).reset_index().rename(columns={"index":"Gene","PC2":"Importance value for PC2"})

In [ ]:
train2.head()

# let's try KMeans for some more intuition

In [ ]:
from sklearn.cluster import KMeans
sumofsq = {}
for k in range(1,15):
    km = KMeans(n_clusters=k,init='k-means++',max_iter=1000)
    km = km.fit(X_scl)
    sumofsq[k] = km.inertia_

In [ ]:
sns.set_style('whitegrid')
plt.figure(figsize=(14,8))
plt.xlabel('Number of Clusters(k)')
plt.ylabel('Sum of Square Distances')
plt.title('Elbow Method For Optimal number of Clusters')
sns.pointplot(x=list(sumofsq.keys()),y=list(sumofsq.values()),color='red')
plt.show()

# clusters = 3 seems fine !

In [ ]:
kmeansmodel = KMeans(n_clusters=3, init= 'k-means++', max_iter= 1000)
kmeansmodel.fit_transform(X_scl)
print()

In [ ]:
Kmeansdf = X.copy()
Kmeansdf['Cluster_by_KMeans'] = kmeansmodel.labels_

In [ ]:
Xpcadf['Kmeans_cluster'] = kmeansmodel.labels_

In [ ]:
plt.figure(figsize=(12,7))
sns.set_style("whitegrid")
ax = sns.scatterplot(Xpcadf['PC1'],Xpcadf['PC2'],hue=Xpcadf['Kmeans_cluster'],palette="gist_rainbow")

plt.xlabel(f'PC1 ({round(pca.explained_variance_ratio_[0] * 100,1)} %)')
plt.ylabel(f'PC2 ({round(pca.explained_variance_ratio_[1] * 100,1)} %)')
plt.title('scatter plot of PC1 and PC2 with KMeans clusters')
plt.grid()
plt.show()

In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm

In [ ]:
range_n_clusters = [2, 3, 4, 5, 6,7,8,9,10]
X = X_scl.copy()

for n_clusters in range_n_clusters:
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)
    ax1.set_xlim([-0.1, 1])
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])
    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(X)
    silhouette_avg = silhouette_score(X, cluster_labels)
    print("For n_clusters =", n_clusters,"The average silhouette_score is :", silhouette_avg)
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        y_lower = y_upper + 10

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    ax2.scatter(X[:, 0], X[:, 1], marker='.', s=30, lw=0, alpha=0.7,
                c=colors, edgecolor='k')

    centers = clusterer.cluster_centers_
    ax2.scatter(centers[:, 0], centers[:, 1], marker='o',
                c="white", alpha=1, s=200, edgecolor='k')

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,
                    s=50, edgecolor='k')

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

    plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                  "with n_clusters = %d" % n_clusters),
                 fontsize=14, fontweight='bold')

plt.show()

# From both Elbow method and silhouette analysis n_clusters = 3 seems fine !